In [12]:
import autokeras as ak
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13011325291180170216
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11126124205369387315
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8552537175052242631
physical_device_desc: "device: XLA_GPU device"
]


In [14]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [15]:
#Train
#Признаки, связанные с суммой покупки
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Количество в каждой категории
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Сумма в каждой категории
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Минимум в каждой категории
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

#Максимум в каждой категории
max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Средняя сумма в каждой категории
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Соединим все в одну таблицу
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [16]:
train_data = train.head(1000)
test_data = train.tail(1000)

In [17]:
import time
start_time = time.time()

In [18]:
model_finder = ak.StructuredDataClassifier(overwrite=True, max_trials=50)
X_train = train_data.drop('bins', 1)
y_train = train_data['bins']
X_test = test_data.drop('bins', 1)
y_test = test_data['bins']

In [19]:
model_finder.fit(X_train, y_train)

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 1.8110 - accuracy: 0.28 - ETA: 0s - loss: 1.6689 - accuracy: 0.23 - ETA: 0s - loss: 1.5840 - accuracy: 0.25 - ETA: 0s - loss: 1.5463 - accuracy: 0.28 - ETA: 0s - loss: 1.5439 - accuracy: 0.29 - 6s 215ms/step - loss: 1.5372 - accuracy: 0.3017 - val_loss: 1.4119 - val_accuracy: 0.2798
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 1.5300 - accuracy: 0.40 - ETA: 0s - loss: 1.3971 - accuracy: 0.40 - ETA: 0s - loss: 1.3657 - accuracy: 0.39 - ETA: 0s - loss: 1.3289 - accuracy: 0.39 - ETA: 0s - loss: 1.3278 - accuracy: 0.40 - 1s 42ms/step - loss: 1.3222 - accuracy: 0.4111 - val_loss: 1.3250 - val_accuracy: 0.3095
Epoch 3/1000
26/26 [==============================] - ETA: 0s - loss: 1.2215 - accuracy: 0.53 - ETA: 0s - loss: 1.3305 - accuracy: 0.41 - ETA: 0s - loss: 1.2740 - accuracy: 0.43 - ETA: 0s - loss: 1.2696 - accuracy: 0.43 - ETA: 0s - loss: 1.2663 - accuracy: 0.42 - 1s 41ms/step - loss: 1.2549 -

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 1.3770 - accuracy: 0.34 - ETA: 0s - loss: 1.4694 - accuracy: 0.33 - ETA: 0s - loss: 1.4064 - accuracy: 0.37 - ETA: 0s - loss: 1.3679 - accuracy: 0.38 - ETA: 0s - loss: 1.3691 - accuracy: 0.38 - 5s 206ms/step - loss: 1.3541 - accuracy: 0.3978 - val_loss: 1.6135 - val_accuracy: 0.2976
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 0.9342 - accuracy: 0.53 - ETA: 0s - loss: 1.0816 - accuracy: 0.54 - ETA: 0s - loss: 1.0582 - accuracy: 0.54 - ETA: 0s - loss: 1.0475 - accuracy: 0.55 - ETA: 0s - loss: 1.0661 - accuracy: 0.54 - 1s 42ms/step - loss: 1.0521 - accuracy: 0.5541 - val_loss: 1.2810 - val_accuracy: 0.4048
Epoch 3/1000
26/26 [==============================] - ETA: 0s - loss: 0.9239 - accuracy: 0.62 - ETA: 0s - loss: 1.0901 - accuracy: 0.54 - ETA: 0s - loss: 1.0091 - accuracy: 0.56 - ETA: 0s - loss: 0.9873 - accuracy: 0.58 - ETA: 0s - loss: 0.9990 - accuracy: 0.56 - 1s 41ms/step - loss: 0.9903 -

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 1.5620 - accuracy: 0.28 - ETA: 0s - loss: 1.6707 - accuracy: 0.26 - ETA: 0s - loss: 1.5598 - accuracy: 0.31 - ETA: 0s - loss: 1.4749 - accuracy: 0.33 - ETA: 0s - loss: 1.4555 - accuracy: 0.32 - 5s 207ms/step - loss: 1.4217 - accuracy: 0.3365 - val_loss: 10.9950 - val_accuracy: 0.2857
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 1.2711 - accuracy: 0.37 - ETA: 0s - loss: 1.2612 - accuracy: 0.40 - ETA: 0s - loss: 1.2698 - accuracy: 0.39 - ETA: 0s - loss: 1.2402 - accuracy: 0.41 - ETA: 0s - loss: 1.2128 - accuracy: 0.43 - 1s 41ms/step - loss: 1.2257 - accuracy: 0.4339 - val_loss: 2.4782 - val_accuracy: 0.3214
Epoch 3/1000
26/26 [==============================] - ETA: 0s - loss: 1.2263 - accuracy: 0.40 - ETA: 0s - loss: 1.2586 - accuracy: 0.45 - ETA: 0s - loss: 1.2524 - accuracy: 0.44 - ETA: 0s - loss: 1.2329 - accuracy: 0.44 - ETA: 0s - loss: 1.1998 - accuracy: 0.45 - ETA: 0s - loss: 1.1938 - acc

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 1048.6772 - accuracy: 0.250 - ETA: 0s - loss: 1839.0916 - accuracy: 0.262 - ETA: 0s - loss: 1510.8984 - accuracy: 0.302 - ETA: 0s - loss: 1229.7312 - accuracy: 0.317 - ETA: 0s - loss: 1135.9750 - accuracy: 0.318 - ETA: 0s - loss: 1063.0786 - accuracy: 0.334 - ETA: 0s - loss: 963.9066 - accuracy: 0.330 - 6s 242ms/step - loss: 926.6249 - accuracy: 0.3293 - val_loss: 253.6442 - val_accuracy: 0.3155
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 421.5912 - accuracy: 0.31 - ETA: 0s - loss: 311.8601 - accuracy: 0.29 - ETA: 0s - loss: 262.2856 - accuracy: 0.31 - ETA: 0s - loss: 231.7252 - accuracy: 0.31 - ETA: 0s - loss: 207.2283 - accuracy: 0.32 - ETA: 0s - loss: 198.7807 - accuracy: 0.32 - ETA: 0s - loss: 184.2041 - accuracy: 0.31 - 2s 59ms/step - loss: 179.6650 - accuracy: 0.3149 - val_loss: 20.0591 - val_accuracy: 0.3214
Epoch 3/1000
26/26 [==============================] - ETA: 0s - loss: 52.4166

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 4159.8564 - accuracy: 0.156 - ETA: 0s - loss: 2447.3513 - accuracy: 0.236 - ETA: 0s - loss: 1827.0377 - accuracy: 0.271 - ETA: 0s - loss: 1415.5703 - accuracy: 0.286 - ETA: 0s - loss: 1187.5701 - accuracy: 0.310 - 6s 215ms/step - loss: 1166.0924 - accuracy: 0.3065 - val_loss: 425.6121 - val_accuracy: 0.3452
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 374.9932 - accuracy: 0.34 - ETA: 0s - loss: 270.9118 - accuracy: 0.41 - ETA: 0s - loss: 251.4987 - accuracy: 0.43 - ETA: 0s - loss: 261.2348 - accuracy: 0.42 - ETA: 0s - loss: 263.9739 - accuracy: 0.42 - 1s 49ms/step - loss: 263.1353 - accuracy: 0.4195 - val_loss: 273.6231 - val_accuracy: 0.3810
Epoch 3/1000
26/26 [==============================] - ETA: 0s - loss: 217.2300 - accuracy: 0.37 - ETA: 0s - loss: 180.9202 - accuracy: 0.44 - ETA: 0s - loss: 159.8790 - accuracy: 0.48 - ETA: 0s - loss: 157.3683 - accuracy: 0.50 - ETA: 0s - loss: 146.9317

...

26/26 [==============================] - ETA: 0s - loss: 1.2662 - accuracy: 0.34 - ETA: 0s - loss: 1.3105 - accuracy: 0.40 - ETA: 0s - loss: 1.3355 - accuracy: 0.40 - ETA: 0s - loss: 1.2941 - accuracy: 0.41 - ETA: 0s - loss: 1.2827 - accuracy: 0.41 - 2s 58ms/step - loss: 1.2772 - accuracy: 0.4279 - val_loss: 1.2284 - val_accuracy: 0.4286
Epoch 89/1000
26/26 [==============================] - ETA: 0s - loss: 1.2955 - accuracy: 0.46 - ETA: 0s - loss: 1.3251 - accuracy: 0.41 - ETA: 0s - loss: 1.2702 - accuracy: 0.45 - ETA: 0s - loss: 1.2606 - accuracy: 0.45 - ETA: 0s - loss: 1.2354 - accuracy: 0.46 - 1s 47ms/step - loss: 1.2426 - accuracy: 0.4567 - val_loss: 1.2378 - val_accuracy: 0.4226
Epoch 90/1000
26/26 [==============================] - ETA: 0s - loss: 1.2801 - accuracy: 0.40 - ETA: 0s - loss: 1.2803 - accuracy: 0.39 - ETA: 0s - loss: 1.2681 - accuracy: 0.43 - ETA: 0s - loss: 1.2617 - accuracy: 0.42 - ETA: 0s - loss: 1.2448 - accuracy: 0.43 - ETA: 0s - loss: 1.2469 - accuracy: 0.42 -

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 1.6225 - accuracy: 0.37 - ETA: 0s - loss: 1.7063 - accuracy: 0.32 - ETA: 0s - loss: 1.5993 - accuracy: 0.36 - ETA: 0s - loss: 1.6047 - accuracy: 0.37 - ETA: 0s - loss: 1.5898 - accuracy: 0.38 - ETA: 0s - loss: 1.5923 - accuracy: 0.37 - ETA: 0s - loss: 1.5664 - accuracy: 0.38 - 6s 234ms/step - loss: 1.5519 - accuracy: 0.3882 - val_loss: 1.9122 - val_accuracy: 0.3512
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 0.9901 - accuracy: 0.59 - ETA: 0s - loss: 1.1119 - accuracy: 0.55 - ETA: 0s - loss: 1.0728 - accuracy: 0.56 - ETA: 0s - loss: 1.0459 - accuracy: 0.58 - ETA: 0s - loss: 1.0571 - accuracy: 0.55 - ETA: 0s - loss: 1.0736 - accuracy: 0.55 - ETA: 0s - loss: 1.1162 - accuracy: 0.53 - 2s 58ms/step - loss: 1.1031 - accuracy: 0.5469 - val_loss: 1.4025 - val_accuracy: 0.3750
Epoch 3/1000
26/26 [==============================] - ETA: 0s - loss: 0.6572 - accuracy: 0.78 - ETA: 0s - loss: 1.0205 - accu

26/26 [==============================] - ETA: 0s - loss: 0.3296 - accuracy: 0.87 - ETA: 0s - loss: 0.3876 - accuracy: 0.87 - ETA: 0s - loss: 0.4996 - accuracy: 0.85 - ETA: 0s - loss: 0.4308 - accuracy: 0.88 - ETA: 0s - loss: 0.4198 - accuracy: 0.86 - ETA: 0s - loss: 0.4230 - accuracy: 0.86 - ETA: 0s - loss: 0.4231 - accuracy: 0.86 - 1s 48ms/step - loss: 0.4067 - accuracy: 0.8690 - val_loss: 1.2633 - val_accuracy: 0.5357
Epoch 20/1000
26/26 [==============================] - ETA: 0s - loss: 0.4679 - accuracy: 0.90 - ETA: 0s - loss: 0.4314 - accuracy: 0.87 - ETA: 0s - loss: 0.4286 - accuracy: 0.87 - ETA: 0s - loss: 0.4156 - accuracy: 0.87 - ETA: 0s - loss: 0.4030 - accuracy: 0.87 - ETA: 0s - loss: 0.4150 - accuracy: 0.86 - ETA: 0s - loss: 0.4046 - accuracy: 0.86 - 1s 44ms/step - loss: 0.3949 - accuracy: 0.8690 - val_loss: 1.2814 - val_accuracy: 0.5536


INFO:tensorflow:Oracle triggered exit
Epoch 1/34
32/32 [==============================] - ETA: 0s - loss: 1.5850 - accuracy: 0.25 - ETA: 0s - loss: 1.9219 - accuracy: 0.24 - ETA: 0s - loss: 1.8173 - accuracy: 0.24 - ETA: 0s - loss: 1.7685 - accuracy: 0.25 - ETA: 0s - loss: 1.7143 - accuracy: 0.26 - ETA: 0s - loss: 1.6882 - accuracy: 0.25 - ETA: 0s - loss: 1.6638 - accuracy: 0.27 - 0s 11ms/step - loss: 1.6591 - accuracy: 0.2760
Epoch 2/34
32/32 [==============================] - ETA: 0s - loss: 1.3322 - accuracy: 0.40 - ETA: 0s - loss: 1.5056 - accuracy: 0.34 - ETA: 0s - loss: 1.4695 - accuracy: 0.34 - ETA: 0s - loss: 1.4427 - accuracy: 0.35 - ETA: 0s - loss: 1.4457 - accuracy: 0.35 - ETA: 0s - loss: 1.4389 - accuracy: 0.34 - ETA: 0s - loss: 1.4347 - accuracy: 0.34 - 0s 11ms/step - loss: 1.4383 - accuracy: 0.3470
Epoch 3/34
32/32 [==============================] - ETA: 0s - loss: 1.2208 - accuracy: 0.34 - ETA: 0s - loss: 1.3655 - accuracy: 0.33 - ETA: 0s - loss: 1.4036 - accuracy: 0.31 

32/32 [==============================] - ETA: 0s - loss: 1.0922 - accuracy: 0.53 - ETA: 0s - loss: 0.9244 - accuracy: 0.65 - ETA: 0s - loss: 0.8949 - accuracy: 0.65 - ETA: 0s - loss: 0.8929 - accuracy: 0.64 - ETA: 0s - loss: 0.8934 - accuracy: 0.63 - ETA: 0s - loss: 0.8838 - accuracy: 0.64 - ETA: 0s - loss: 0.8893 - accuracy: 0.64 - 0s 11ms/step - loss: 0.8831 - accuracy: 0.6470
Epoch 22/34
32/32 [==============================] - ETA: 0s - loss: 1.0031 - accuracy: 0.56 - ETA: 0s - loss: 0.9333 - accuracy: 0.60 - ETA: 0s - loss: 0.8810 - accuracy: 0.61 - ETA: 0s - loss: 0.8917 - accuracy: 0.60 - ETA: 0s - loss: 0.8828 - accuracy: 0.62 - ETA: 0s - loss: 0.8704 - accuracy: 0.62 - ETA: 0s - loss: 0.8759 - accuracy: 0.62 - ETA: 0s - loss: 0.8799 - accuracy: 0.62 - 0s 12ms/step - loss: 0.8799 - accuracy: 0.6220
Epoch 23/34
32/32 [==============================] - ETA: 0s - loss: 0.8681 - accuracy: 0.59 - ETA: 0s - loss: 0.9004 - accuracy: 0.60 - ETA: 0s - loss: 0.8641 - accuracy: 0.62 - ETA

In [ ]:
model_finder.evaluate(X_test, y_test)

In [ ]:
end_time = time.time()

In [ ]:
print("Time spent", end_time - start_time)